# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [3]:
LOCAL_DEV = True # to switch between developing locally and on colab

if not LOCAL_DEV:
    # TODO: need to upload data files on Google Drive?
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
#Imports
import numpy as np
import torch
import pandas as pd

In [5]:
#visualising training data
if LOCAL_DEV:
    train = pd.read_json("../data/train-claims.json") # for local dev
    
else:
    train = pd.read_json("/content/drive/MyDrive/data/train-claims.json") # on colab
train = train.transpose()
train.head()

,claim_text,claim_label,evidences
claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1..."
claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]"
claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]"
claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5..."
claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72..."


In [6]:
#visualising evidence data
#visualising evidence data
if LOCAL_DEV:
    evidence = pd.read_json("../data/evidence.json",typ='series')
else:
    evidence = pd.read_json("/content/drive/MyDrive/data/evidence.json",typ='series')

In [7]:
print(len(evidence))
evidence.head()

1208827


evidence-0    John Bennet Lawes, English entrepreneur and ag...
evidence-1    Lindberg began his professional career at the ...
evidence-2    ``Boston (Ladies of Cambridge)'' by Vampire We...
evidence-3    Gerald Francis Goyer (born October 20, 1936) w...
evidence-4    He detected abnormalities of oxytocinergic fun...
dtype: object

In [8]:
if LOCAL_DEV:
    test = pd.read_json("../data/test-claims-unlabelled.json")
else:
    test = pd.read_json("/content/drive/MyDrive/data/test-claims-unlabelled.json")
test = test.transpose()
test.head()


,claim_text
claim-2967,The contribution of waste heat to the global c...
claim-979,“Warm weather worsened the most recent five-ye...
claim-1609,Greenland has only lost a tiny fraction of its...
claim-1020,“The global reef crisis does not necessarily m...
claim-2599,Small amounts of very active substances can ca...


In [9]:
#preprocessing
# punctuations should be removed, common words such as the, is, are, should be removed. all words also should be lemmentised and stemmed.


In [10]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [11]:
import string
import contractions
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kaiyuancui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kaiyuancui/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaiyuancui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:


def preprocess_data(data: pd.Series, limit=10000) -> pd.Series:
  preprocessed_data = {}
  stop_words = set(stopwords.words('english'))
  stop_words.remove('not')
  count = 0
  for id, text in data.items():
    text = text.lower()
    text = contractions.fix(text)
    tokens = word_tokenize(text)
    wnl = WordNetLemmatizer()
    lemmatized_tokens = [wnl.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    preprocessed_data[id] = " ".join(lemmatized_tokens)
    count += 1
    if count >= limit:
      break

  return pd.Series(preprocessed_data)

processed_evidence = preprocess_data(evidence)

test_claims = test['claim_text']
processed_test = preprocess_data(test_claims)
processed_test.head()


claim-2967               contribution waste heat global climate
claim-979     warm weather worsened recent drought included ...
claim-1609                greenland lost tiny fraction ice mass
claim-1020    global reef crisis not necessarily mean extinc...
claim-2599     small amount active substance cause large effect
dtype: object

In [13]:
processed_evidence = processed_evidence[processed_evidence.str.strip().str.len() > 0]
processed_evidence.head()

evidence-0    john bennet lawes english entrepreneur agricul...
evidence-1    lindberg began professional career age eventua...
evidence-2                boston lady cambridge vampire weekend
evidence-3    gerald francis goyer born october professional...
evidence-4    detected abnormality oxytocinergic function sc...
dtype: object

# Two steps for the this task
# first. find all relavent evidence, either use contextual embedding or similarity scoring
# second. classify the evidents into 4 classes.

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Flatten, Dense

In [15]:
# Vectorizing preprocessed text
# vectorizer = TfidfVectorizer()
# all_texts = pd.concat([processed_evidence, processed_test])
# vectorizer.fit(all_texts)
# evidence_tfidf = vectorizer.transform(processed_evidence)
# test_tfidf = vectorizer.transform(processed_test)

# vectorizer.get_feature_names_out()
# print(test_tfidf[0])
# #print(test_tfidf.shape)



# get word embeddings

# Sample text data
# corpus = ["I like to eat pizza", "Pizza is my favorite food", "I enjoy eating pizza with friends"]

corpus = pd.concat([processed_evidence, processed_test])

# Tokenization and vocabulary creation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
vocab = tokenizer.word_index
print(vocab)
vocab_size = len(tokenizer.word_index) + 1

# Data encoding
sequences = tokenizer.texts_to_sequences(corpus)
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Define the neural network
embedding_dim = 100  # Dimensionality of the word embeddings
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='linear'))  # Output layer, could be adjusted based on task



# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Create dummy target data
dummy_target = np.zeros((len(padded_sequences), 1))  # Dummy target data of zeros

# Train the model with dummy target data
model.fit(padded_sequences, dummy_target, epochs=10, batch_size=32)

# Get the learned word embeddings
word_embeddings = model.layers[0].get_weights()[0]

#print(word_embeddings)




{'also': 1, 'state': 2, 'born': 3, 'one': 4, 'first': 5, 'new': 6, 'year': 7, 'district': 8, 'known': 9, 'american': 10, 'county': 11, 'world': 12, 'two': 13, 'united': 14, 'national': 15, 'school': 16, 'film': 17, 'not': 18, 'city': 19, 'time': 20, 'album': 21, 'may': 22, 'member': 23, 'part': 24, 'area': 25, 'season': 26, 'university': 27, 'south': 28, 'located': 29, 'team': 30, 'village': 31, 'family': 32, 'work': 33, 'name': 34, 'station': 35, 'football': 36, 'second': 37, 'series': 38, 'group': 39, 'used': 40, 'north': 41, 'game': 42, 'released': 43, 'october': 44, 'played': 45, 'song': 46, 'club': 47, 'system': 48, 'league': 49, 'since': 50, 'company': 51, 'many': 52, 'music': 53, 'band': 54, 'town': 55, 'specie': 56, 'former': 57, 'including': 58, 'km': 59, 'house': 60, 'war': 61, 'de': 62, 'three': 63, 'event': 64, 'woman': 65, 'high': 66, 'population': 67, 'west': 68, 'people': 69, 'would': 70, 'later': 71, 'community': 72, 'international': 73, 'record': 74, 'college': 75, 'pl

In [35]:
import numpy as np
import re
np.random.seed(42)

# replace with our corpus
text = '''Machine learning is the study of computer algorithms that \
improve automatically through experience. It is seen as a \
subset of artificial intelligence. Machine learning algorithms \
build a mathematical model based on sample data, known as \
training data, in order to make predictions or decisions without \
being explicitly programmed to do so. Machine learning algorithms \
are used in a wide variety of applications, such as email filtering \
and computer vision, where it is difficult or infeasible to develop \
conventional algorithms to perform the needed tasks.''' 


def tokenize(text):
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())

tokens = tokenize(text)
print(tokens)

# def mapping(tokens):
#     word_to_id = {}
#     id_to_word = {}
    
#     for i, token in enumerate(set(tokens)):
#         word_to_id[token] = i
#         id_to_word[i] = token
    
#     return word_to_id, id_to_word

# word_to_id, id_to_word = mapping(tokens)
# print(id_to_word)
# word_to_id


word_to_id = tokenizer.word_index
id_to_word = tokenizer.index_word
tokens = tokenizer.sequences_to_texts(sequences)
print(tokens)

def generate_training_data(tokens, word_to_id, window):
    X = []
    y = []
    n_tokens = len(tokens)
    
    for i in range(n_tokens):
        idx = concat(
            range(max(0, i - window), i), 
            range(i, min(n_tokens, i + window + 1))
        )
        for j in idx:
            if i == j:
                continue
            X.append(one_hot_encode(word_to_id[tokens[i]], len(word_to_id)))
            y.append(one_hot_encode(word_to_id[tokens[j]], len(word_to_id)))
    
    return np.asarray(X), np.asarray(y)


def concat(*iterables):
    for iterable in iterables:
        yield from iterable

def one_hot_encode(id, vocab_size):
    res = [0] * vocab_size
    res[id] = 1
    return res

X, y = generate_training_data(tokens, word_to_id, 2)

def init_network(vocab_size, n_embedding):
    model = {
        "w1": np.random.randn(vocab_size, n_embedding),
        "w2": np.random.randn(n_embedding, vocab_size)
    }
    return model

model = init_network(len(word_to_id), 10)

def forward(model, X, return_cache=True):
    cache = {}
    
    cache["a1"] = X @ model["w1"]
    cache["a2"] = cache["a1"] @ model["w2"]
    cache["z"] = softmax(cache["a2"])
    
    if not return_cache:
        return cache["z"]
    return cache

def softmax(X):
    res = []
    for x in X:
        exp = np.exp(x)
        res.append(exp / exp.sum())
    return res

def backward(model, X, y, alpha):
    cache  = forward(model, X)
    da2 = cache["z"] - y
    dw2 = cache["a1"].T @ da2
    da1 = da2 @ model["w2"].T
    dw1 = X.T @ da1
    assert(dw2.shape == model["w2"].shape)
    assert(dw1.shape == model["w1"].shape)
    model["w1"] -= alpha * dw1
    model["w2"] -= alpha * dw2
    return cross_entropy(cache["z"], y)


def cross_entropy(z, y):
    return - np.sum(np.log(z) * y)

n_iter = 50
learning_rate = 0.05

history = [backward(model, X, y, learning_rate) for _ in range(n_iter)]

learning = one_hot_encode(word_to_id["learning"], len(word_to_id))
result = forward(model, [learning], return_cache=False)[0]

for word in (id_to_word[id] for id in np.argsort(result)[::-1]):
    print(word)


['machine', 'learning', 'is', 'the', 'study', 'of', 'computer', 'algorithms', 'that', 'improve', 'automatically', 'through', 'experience', 'it', 'is', 'seen', 'as', 'a', 'subset', 'of', 'artificial', 'intelligence', 'machine', 'learning', 'algorithms', 'build', 'a', 'mathematical', 'model', 'based', 'on', 'sample', 'data', 'known', 'as', 'training', 'data', 'in', 'order', 'to', 'make', 'predictions', 'or', 'decisions', 'without', 'being', 'explicitly', 'programmed', 'to', 'do', 'so', 'machine', 'learning', 'algorithms', 'are', 'used', 'in', 'a', 'wide', 'variety', 'of', 'applications', 'such', 'as', 'email', 'filtering', 'and', 'computer', 'vision', 'where', 'it', 'is', 'difficult', 'or', 'infeasible', 'to', 'develop', 'conventional', 'algorithms', 'to', 'perform', 'the', 'needed', 'tasks']
['john bennet lawes english entrepreneur agricultural scientist', 'lindberg began professional career age eventually moving new york city', 'boston lady cambridge vampire weekend', 'gerald francis g

KeyError: 'john bennet lawes english entrepreneur agricultural scientist'

In [23]:
# Function to find most similar words to a given word
def find_similar_words(word, embeddings, vocab, top_n=5):
    if word not in vocab:
        print(f"Word '{word}' not found in vocabulary.")
        return
    else:
        word_index = vocab[word]
        word_embedding = embeddings[word_index]
        similarities = cosine_similarity([word_embedding], embeddings)[0]
        most_similar_indices = similarities.argsort()[-top_n-1:-1][::-1]  # Exclude the word itself
        most_similar_words = [word for word, idx in vocab.items() if idx in most_similar_indices]
        most_similar_similarities = [similarities[index] for index in most_similar_indices]
        return most_similar_words, most_similar_similarities


# Example usage
word = 'world'
similar_words = find_similar_words(word, word_embeddings, vocab)
print(f"Words similar to '{word}': {similar_words}")

Words similar to 'world': (['regional', 'mall', 'cook', 'loyd', 'koula'], [0.46144778, 0.41037938, 0.4035752, 0.4028944, 0.4018784])


In [17]:




similarity_matrix = cosine_similarity(test_tfidf, evidence_tfidf)

def getTopN(similarity_matrix, test, evidence, n):
  test = test.to_frame(name='claim_text')
  top_indices = np.argsort(-similarity_matrix, axis = 1)[:, :n]
  top_evidence = [[str(evidence.index[i]) for i in row] for row in top_indices]
  test['evidences'] = top_evidence
  return test

test_with_evi = getTopN(similarity_matrix, processed_test, processed_evidence, 5)
test_with_evi.head()




NameError: name 'test_tfidf' is not defined

In [ ]:
#print(train.head())
#print(test_with_evi.head())
# claim: contribution waste heat global climate
print(evidence['evidence-308923'])
print(evidence['evidence-213569'])

#greenland lost tiny fraction ice mass	
print(evidence['evidence-962481'])
print(evidence['evidence-1200633'])



In [ ]:
from torch import nn, optim

# define hypermeter
sequence_len = 28
input_len = 28
hidden_size = 128
num_layers = 2
num_classes = 4
num_epchos = 5
learning_rate = 0.01


In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_length, hidden_size, num_classes, num_layers):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_len, hidden_size, num_layers, num_classes, batch_first=True)
    self.output_layer = nn.Linear(hidden_size, num_classes)

  def forward(self, X):
    hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
    cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
    out, _ = self.lstm(X, (hidden_states, cell_states))
    out = self.output_layer(out[:, -1, :])
    return out


In [ ]:
model = LSTM(input_len, hidden_size, num_classes, num_layers)
print(model)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*